Import libraries

In [1]:
import os 
import json
import numpy as np
import torch 
import matplotlib.pyplot as plt

from utils import load_pickle, pickle_data

mkdir -p failed for path /afs/csail.mit.edu/u/k/klingmin/.config/matplotlib: [Errno 13] Permission denied: '/afs/csail.mit.edu/u/k/klingmin/.config/matplotlib'
Matplotlib created a temporary cache directory at /tmp/matplotlib-adup4we1 because there was an issue with the default path (/afs/csail.mit.edu/u/k/klingmin/.config/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Settings

In [6]:
cache_reuslts_folder = "../cache/jaccard"
if not os.path.exists(cache_reuslts_folder): os.makedirs(cache_reuslts_folder)
baseline_results_folder = "/data/rbg/users/klingmin/projects/ML_MS_analysis/FP_prediction/baseline_models/best_models"
MIST_results_folder = "/data/rbg/users/klingmin/projects/ML_MS_analysis/FP_prediction/mist/best_models"

results_folder = [baseline_results_folder, MIST_results_folder]

datasets = ["canopus", "massspecgym", "nist2023"]
splits = ["random", "inchikey_vanilla", "scaffold_vanilla", "LS"]
models = ["binned_", "MS_", "formula_"]

Write the predictions more easily so that we can access it later

In [19]:
for dataset in datasets:

    for split in splits: 

        current_folder = os.path.join(baseline_results_folder, dataset)

        for model in models: 

            output_path = os.path.join(cache_reuslts_folder, f"{dataset}_{model}{split}.pkl")
            if os.path.exists(output_path): continue 
            
            results_path = [f for f in os.listdir(current_folder) if model in f and split in f]
            assert len(results_path) == 1 
            results_path = os.path.join(current_folder, results_path[0], "test_results.pkl")
            results = load_pickle(results_path)
            results = {(k.item() if isinstance(k, torch.Tensor) else k): v["jaccard"] for k, v in results.items()}

            pickle_data(results, output_path)

In [18]:
for dataset in datasets:

    current_folder = os.path.join(MIST_results_folder, dataset)

    for split in splits: 
        
        if split == "LS": current_folder = os.path.join(MIST_results_folder, f"{dataset}_LS")
        
        output_path = os.path.join(cache_reuslts_folder, f"{dataset}_MIST_{split}.pkl")
        if os.path.exists(output_path): continue 
        results_path = [f for f in os.listdir(current_folder) if split in f]
        assert len(results_path) == 1 
        
        results_path = os.path.join(current_folder, results_path[0], "test_results.pkl")
        results = load_pickle(results_path)
        results = {(k.item() if isinstance(k, torch.Tensor) else k): v["jaccard"] for k, v in results.items()}

        pickle_data(results, output_path)

['C_MIST_meta_4096_LS'] LS
['MSG_MIST_4096_random'] random
['MSG_MIST_meta_4096_inchikey_vanilla'] inchikey_vanilla
['MSG_MIST_meta_4096_scaffold_vanilla'] scaffold_vanilla
['MSG_MIST_meta_4096_LS'] LS
['NIST2023_MIST_meta_4096_random'] random
['NIST2023_MIST_4096_inchikey_vanilla'] inchikey_vanilla
['NIST2023_MIST_meta_4096_scaffold_vanilla'] scaffold_vanilla
['NIST2023_MIST_meta_4096_LS'] LS


In [ ]:
cutoffs = np.arange(0, 1.0, 0.01)[::-1]
cutoffs_list = [] 
jaccard_list = [] 

for c in cutoffs:

    if c == 0: continue
    kept_ids = [str(k) for k,v in oov_rate.items() if v < c]
    jaccard = [results[k] for k in kept_ids]

    cutoffs_list.append(c)
    jaccard_list.append(np.mean(jaccard))

plt.plot(cutoffs_list, jaccard_list, marker='o')


In [ ]:
sieved_results = {} 

for k, v in results.items():

    if type(k) == torch.Tensor: k = k.item()
    sieved_results[k] = v["jaccard"]
    
    print(k, v.keys())
    a = z

In [ ]:
sieved_results